In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- 1. ตั้งค่าการแสดงผล ---
pd.set_option('display.max_columns', None)  # โชว์ทุกคอลัมน์ ไม่ซ่อน
sns.set_style('whitegrid')                  # กราฟสวยอ่านง่าย

# --- 2. กำหนด Path (แบบฉลาด) ---
# '../Dataset' แปลว่า "ถอยหลังจาก folder notebooks ไป 1 ขั้น แล้วเข้า folder Dataset"
DATA_DIR = '../Dataset' 

def load_dataset(filename):
    """ฟังก์ชันช่วยโหลดไฟล์ พร้อมเช็ค Error"""
    path = os.path.join(DATA_DIR, filename)
    if not os.path.exists(path):
        print(f"ไม่เจอไฟล์ที่: {path}")
        return None
    return pd.read_csv(path)

# --- 3. โหลดข้อมูล ---
print(" กำลังโหลดข้อมูล...")

# พยายามโหลดไฟล์ train_values (รองรับทั้งชื่อเก่าและชื่อใหม่)
train_values = load_dataset('train_values.csv') 
if train_values is None:
    train_values = load_dataset('train_values_wJZrCmI.csv')

train_labels = load_dataset('train_labels.csv')

# --- 4. ตรวจสอบความเรียบร้อยก่อน Merge ---
if train_values is not None and train_labels is not None:
    print(f" โหลดสำเร็จ!")
    print(f"   Train Values: {train_values.shape}")
    print(f"   Train Labels: {train_labels.shape}")

    # --- 5. Merge Data (รวมร่าง) ---
    df = train_values.merge(train_labels, on='row_id')
    print(f" Merged Data:  {df.shape} (รวมร่างเสร็จสมบูรณ์)")
    
    # โชว์ตัวอย่างข้อมูล
    display(df.head())
    
    # โชว์ประเภทข้อมูลและค่าว่าง
    print("\n--- Data Info ---")
    df.info()
    
else:
    print("\n Error: โหลดไฟล์ไม่ครบ กรุณาเช็คชื่อไฟล์หรือ Path อีกครั้ง")

 กำลังโหลดข้อมูล...
ไม่เจอไฟล์ที่: ../Dataset\train_values.csv
 โหลดสำเร็จ!
   Train Values: (12600, 59)
   Train Labels: (12600, 2)
 Merged Data:  (12600, 60) (รวมร่างเสร็จสมบูรณ์)


,row_id,country,is_urban,age,female,married,religion,relationship_to_hh_head,education_level,literacy,can_add,can_divide,can_calc_percents,can_calc_compounding,employed_last_year,employment_category_last_year,employment_type_last_year,share_hh_income_provided,income_ag_livestock_last_year,income_friends_family_last_year,income_government_last_year,income_own_business_last_year,income_private_sector_last_year,income_public_sector_last_year,num_times_borrowed_last_year,borrowing_recency,formal_savings,informal_savings,cash_property_savings,has_insurance,has_investment,bank_interest_rate,mm_interest_rate,mfi_interest_rate,other_fsp_interest_rate,num_shocks_last_year,avg_shock_strength_last_year,borrowed_for_emergency_last_year,borrowed_for_daily_expenses_last_year,borrowed_for_home_or_biz_last_year,phone_technology,can_call,can_text,can_use_internet,can_make_transaction,phone_ownership,advanced_phone_use,reg_bank_acct,reg_mm_acct,reg_formal_nbfi_account,financially_included,active_bank_user,active_mm_user,active_formal_nbfi_user,active_informal_nbfi_user,nonreg_active_mm_user,num_formal_institutions_last_year,num_informal_institutions_last_year,num_financial_activities_last_year,poverty_probability
0,0,C,False,18,True,True,P,Other,1.0,True,True,True,True,True,False,housewife_or_student,not_working,1.0,False,False,False,False,False,False,0,0,False,False,False,False,False,NaN,NaN,NaN,NaN,0,0.0,False,False,False,0,True,True,False,False,1,False,True,False,False,True,True,False,False,False,False,1,0,1,0.515
1,1,C,True,30,True,True,P,Other,1.0,True,True,True,False,False,False,housewife_or_student,not_working,NaN,False,False,False,False,False,False,0,0,False,False,False,False,False,NaN,NaN,NaN,NaN,0,0.0,False,False,False,1,True,False,False,False,2,False,True,False,False,True,True,False,False,False,False,1,0,0,0.981
2,2,A,False,20,True,True,Q,Spouse,1.0,True,True,True,True,False,True,employed,irregular_seasonal,1.0,False,False,False,False,False,False,1,2,False,False,False,False,False,NaN,NaN,NaN,NaN,0,0.0,False,False,False,1,True,False,False,False,2,False,False,False,False,False,False,False,False,False,False,0,0,0,0.982
3,3,A,False,61,False,True,Q,Head,0.0,False,True,True,False,True,True,employed,self_employed,NaN,False,False,False,False,False,False,0,0,False,False,False,False,False,NaN,NaN,NaN,NaN,0,0.0,False,False,False,0,True,False,False,False,0,False,False,False,False,False,False,False,False,False,False,0,0,0,0.879
4,4,D,False,26,True,True,X,Spouse,1.0,True,False,True,False,False,False,housewife_or_student,not_working,2.0,False,True,False,False,False,False,0,0,False,False,True,False,False,NaN,NaN,NaN,NaN,0,0.0,False,False,False,0,True,True,False,True,1,False,False,False,False,False,False,False,False,False,False,1,0,3,0.796



--- Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Data columns (total 60 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   row_id                                 12600 non-null  int64  
 1   country                                12600 non-null  object 
 2   is_urban                               12600 non-null  bool   
 3   age                                    12600 non-null  int64  
 4   female                                 12600 non-null  bool   
 5   married                                12600 non-null  bool   
 6   religion                               12600 non-null  object 
 7   relationship_to_hh_head                12600 non-null  object 
 8   education_level                        12364 non-null  float64
 9   literacy                               12600 non-null  bool   
 10  can_add                                12600 non-nu

Data Cleaning & Preprocessing

In [2]:
# --- Cell: The FINAL Master Pipeline (Cleaning + Encoding) ---
# รวมทุกกระบวนการไว้ในที่เดียว เพื่อไม่ให้ข้อมูลถูกรีเซ็ตทับกัน

# 1. รีเซ็ตจากข้อมูลดิบ (จุดเริ่มต้นเดียว)
df_clean = df.copy()
print(" เริ่มรัน Pipeline แบบม้วนเดียวจบ...")

# ---------------------------------------------------------
# STEP 1: จัดการ ID (Prevent Leakage)
# ---------------------------------------------------------
if 'row_id' in df_clean.columns:
    df_clean.set_index('row_id', inplace=True)
    print("   - Set row_id as index")

# ---------------------------------------------------------
# STEP 2: จัดการค่าว่าง (Cleaning Missing Values)
# ---------------------------------------------------------
print(" 2. กำลังจัดการค่าว่าง...")

# 2.1 ดอกเบี้ย -> เติม 0
interest_cols = ['bank_interest_rate', 'mm_interest_rate', 'mfi_interest_rate', 'other_fsp_interest_rate']
df_clean[interest_cols] = df_clean[interest_cols].fillna(0)

# 2.2 รายได้ -> เติม Median
if 'share_hh_income_provided' in df_clean.columns:
    df_clean['share_hh_income_provided'] = df_clean['share_hh_income_provided'].fillna(df_clean['share_hh_income_provided'].median())

# 2.3 การศึกษา -> เติม Mode
if 'education_level' in df_clean.columns:
    df_clean['education_level'] = df_clean['education_level'].fillna(df_clean['education_level'].mode()[0])

print(f"   - เหลือค่าว่าง: {df_clean.isnull().sum().sum()} ตัว (ควรเป็น 0)")

# ---------------------------------------------------------
# STEP 3: แปลงข้อมูล (Transformation)
# ---------------------------------------------------------
print(" 3. กำลังแปลงข้อมูล (Boolean & One-Hot)...")

# 3.1 แปลง Boolean (True/False -> 1/0)
bool_cols = df_clean.select_dtypes(include=['bool']).columns
if len(bool_cols) > 0:
    df_clean[bool_cols] = df_clean[bool_cols].astype(int)

# 3.2 แปลง Object (One-Hot Encoding)
# หาคอลัมน์ที่เป็นตัวหนังสือทั้งหมด
obj_cols = df_clean.select_dtypes(include=['object']).columns
if len(obj_cols) > 0:
    df_clean = pd.get_dummies(df_clean, columns=obj_cols, prefix=obj_cols, dtype=int)

# ---------------------------------------------------------
# STEP 4: ตรวจสอบครั้งสุดท้าย (Final Check)
# ---------------------------------------------------------
print("\n" + "="*40)
print(" สรุปผลลัพธ์สุดท้าย:")
total_missing = df_clean.isnull().sum().sum()
non_numeric = df_clean.select_dtypes(exclude=['number']).columns

if total_missing == 0 and len(non_numeric) == 0:
    print(" ยินดีด้วย! ข้อมูลสะอาด 100% (ไม่มีค่าว่าง + เป็นตัวเลขหมดแล้ว)")
    print(f"Shape: {df_clean.shape}")
    display(df_clean.head())
    
    # Optional: Save ไฟล์เลยถ้ามั่นใจ
    # df_clean.to_csv('cleaned_data.csv')
else:
    print(f" ยังเหลือปัญหา: Missing={total_missing}, Non-Numeric={len(non_numeric)}")
    if total_missing > 0:
        print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])
print("="*40)

 เริ่มรัน Pipeline แบบม้วนเดียวจบ...
   - Set row_id as index
 2. กำลังจัดการค่าว่าง...
   - เหลือค่าว่าง: 0 ตัว (ควรเป็น 0)
 3. กำลังแปลงข้อมูล (Boolean & One-Hot)...

 สรุปผลลัพธ์สุดท้าย:
 ยินดีด้วย! ข้อมูลสะอาด 100% (ไม่มีค่าว่าง + เป็นตัวเลขหมดแล้ว)
Shape: (12600, 83)


,is_urban,age,female,married,education_level,literacy,can_add,can_divide,can_calc_percents,can_calc_compounding,employed_last_year,share_hh_income_provided,income_ag_livestock_last_year,income_friends_family_last_year,income_government_last_year,income_own_business_last_year,income_private_sector_last_year,income_public_sector_last_year,num_times_borrowed_last_year,borrowing_recency,formal_savings,informal_savings,cash_property_savings,has_insurance,has_investment,bank_interest_rate,mm_interest_rate,mfi_interest_rate,other_fsp_interest_rate,num_shocks_last_year,avg_shock_strength_last_year,borrowed_for_emergency_last_year,borrowed_for_daily_expenses_last_year,borrowed_for_home_or_biz_last_year,phone_technology,can_call,can_text,can_use_internet,can_make_transaction,phone_ownership,advanced_phone_use,reg_bank_acct,reg_mm_acct,reg_formal_nbfi_account,financially_included,active_bank_user,active_mm_user,active_formal_nbfi_user,active_informal_nbfi_user,nonreg_active_mm_user,num_formal_institutions_last_year,num_informal_institutions_last_year,num_financial_activities_last_year,poverty_probability,country_A,country_C,country_D,country_F,country_G,country_I,country_J,religion_N,religion_O,religion_P,religion_Q,religion_X,relationship_to_hh_head_Father/Mother,relationship_to_hh_head_Head,relationship_to_hh_head_Other,relationship_to_hh_head_Sister/Brother,relationship_to_hh_head_Son/Daughter,relationship_to_hh_head_Spouse,relationship_to_hh_head_Unknown,employment_category_last_year_employed,employment_category_last_year_housewife_or_student,employment_category_last_year_other,employment_category_last_year_retired_or_disabled,employment_category_last_year_unemployed,employment_type_last_year_irregular_seasonal,employment_type_last_year_not_working,employment_type_last_year_other,employment_type_last_year_salaried,employment_type_last_year_self_employed
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,18,1,1,1.0,1,1,1,1,1,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0.515,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
1,1,30,1,1,1.0,1,1,1,0,0,0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,1,1,0,0,0,2,0,1,0,0,1,1,0,0,0,0,1,0,0,0.981,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,0,20,1,1,1.0,1,1,1,1,0,1,1.0,0,0,0,0,0,0,1,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,1,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.982,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0
3,0,61,0,1,0.0,0,1,1,0,1,1,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.879,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,0,26,1,1,1.0,1,0,1,0,0,0,2.0,0,1,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,3,0.796,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0


In [3]:
# --- Cell: Save Final Data ---

import os
from pathlib import Path

# กำหนดชื่อไฟล์ปลายทาง (ตั้งชื่อให้สื่อความหมาย)
output_filename = 'train_data_cleaned_v1.csv'

# ใช้ Path แบบฉลาดๆ (ถอยจาก notebooks -> ไปที่ Dataset)
# หรือถ้าคุณรันใน Script ก็ใช้ logic เดิมได้เลย
save_path = Path('../Dataset') / output_filename 

print(f" กำลังบันทึกไฟล์ไปที่: {save_path.resolve()}")

# สั่ง Save (index=True เพราะเราต้องการเก็บ row_id ที่เป็น index ไว้ด้วย)
try:
    df_clean.to_csv(save_path, index=True)
    print(f" บันทึกสำเร็จ! ขนาดไฟล์: {save_path.stat().st_size / 1024:.2f} KB")
except Exception as e:
    print(f" Error ในการบันทึก: {e}")

 กำลังบันทึกไฟล์ไปที่: C:\Poverty_prediction_capstone\Dataset\train_data_cleaned_v1.csv
 บันทึกสำเร็จ! ขนาดไฟล์: 2369.34 KB
